In [1]:
import numpy as np
import sys

import tensorflow as tf
from tensorflow.keras.layers import (Input, Layer, Dense, Lambda, 
                                     Dropout, Multiply, BatchNormalization, 
                                     Reshape, Concatenate, Conv2D, Permute)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Resizing

from tensorflow.keras.datasets import cifar10

import os

#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

# Train Model to Be Explained

### Save Dir

In [3]:
model_dir = os.path.join(os.getcwd(), 'cifar_models')
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

### Parameters

In [4]:
BATCH_SIZE = 32
EPOCHS = 100
LR = 1e-3
INPUT_SHAPE = (224, 224, 3)

### Load Data

In [5]:
from sklearn.model_selection import train_test_split

num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, train_size=0.5, random_state=420)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
#Resize to 224x224

print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Make TF Dataset
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

50000 train samples
5000 val samples
5000 test samples


### Batch Data

In [6]:
def batch_data(dataset, fn, batch_size=32):
    dataset = dataset.map(fn)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

### Reformat Data

In [7]:
def reformat(x, y):
    
    x = tf.cast(x, tf.float32)
    x = Resizing(INPUT_SHAPE[0], INPUT_SHAPE[1], interpolation='nearest')(x)
    x = tf.keras.applications.resnet50.preprocess_input(x)
    
    return (x, y)

ds_train = batch_data(ds_train, reformat, BATCH_SIZE)
ds_val = batch_data(ds_val, reformat, BATCH_SIZE)
ds_test = batch_data(ds_test, reformat, BATCH_SIZE)

### Model

In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50

base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, name='input')

net = base_model(model_input)
out = Dense(num_classes, activation='softmax')(net)

model = Model(model_input, out)

# Metrics
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

# Model Checkpointing
model_weights_path = os.path.join(model_dir, 'model_weights.h5')
checkpoint = ModelCheckpoint(model_weights_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only=True)

# LR Schedule
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=3, 
                             verbose=1, mode='min', cooldown=1, min_lr=1e-4)

# Early Stopping 
earlyStop = EarlyStopping(monitor="val_loss", mode="min", patience=10) 

# Compile Model
CALLBACKS = [checkpoint, earlyStop, reduceLR]
OPTIMIZER = tf.keras.optimizers.Adam(LR)

model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)
    
if os.path.isfile(os.path.join(model_dir, 'model_weights.h5')):
    
    print('Loading saved model')
    model.load_weights(model_weights_path)
    model.trainable = False

    # Evaluate
    model.evaluate(ds_test)
    
else:
    # Train Model
    model.fit(ds_train,
              epochs = EPOCHS,
              validation_data = ds_val,
              callbacks = CALLBACKS)

    # Get Checkpointed Model
    model.load_weights(model_weights_path)
    model.trainable = False

    # Evaluate
    model.evaluate(ds_test)

Loading saved model
157/157 [==============================] - 14s 87ms/step - loss: 0.4799 - auroc: 0.9817 - auprc: 0.9419 - accuracy: 0.8934


# Explain w/ FastSHAP

### Specify Explainer Architecture

In [9]:
model_input = Input(shape=INPUT_SHAPE, name='input')

base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE
)
base_model = Model(base_model.input, base_model.get_layer('conv4_block3_2_conv').output)
base_model.trainable = True

net = base_model(model_input)

# Learn Phi 
phi = Conv2D(num_classes, 1)(net)

explainer = Model(model_input, phi)

### Extract Superpixel Size (From Explainer Model Output)

In [10]:
superpixel_size = int(INPUT_SHAPE[1]/explainer.output.shape[1])

## Train Surrogate Model

In [11]:
from fastshap import ImageSurrogate

### Surrogate Model

In [12]:
base_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
)
base_model.trainable = True

model_input = Input(shape=INPUT_SHAPE, name='input')

net = base_model(model_input)
out = Dense(num_classes, activation='softmax')(net)

surrogate_model = Model(model_input, out)

### Train

In [13]:
if os.path.isfile(os.path.join(model_dir, 'surrogate.h5')):
    print('Loading saved model')
    surrogate_model = tf.keras.models.load_model(os.path.join(model_dir, 'surrogate.h5'))
    
    surrogate = ImageSurrogate(surrogate_model = surrogate_model,
                               baseline = 0,
                               width = INPUT_SHAPE[1], 
                               height = INPUT_SHAPE[0], 
                               superpixel_size = superpixel_size)
else:    
    surrogate = ImageSurrogate(surrogate_model = surrogate_model,
                               baseline = 0,
                               width = INPUT_SHAPE[1], 
                               height = INPUT_SHAPE[0], 
                               superpixel_size = superpixel_size)

    surrogate.train(original_model = model,
                    train_data = ds_train,
                    val_data = ds_val,
                    batch_size = 32,
                    max_epochs = 100,
                    validation_batch_size = 32,
                    loss_fn='categorical_crossentropy',
                    lr=1e-3,
                    min_lr=1e-5,
                    lr_factor=0.9,
                    lookback=10,
                    gpu_device=0,
                    verbose=1, 
                    model_dir = model_dir)

Loading saved model


## Train FastSHAP

In [15]:
from fastshap import FastSHAP, EfficiencyNormalization

In [16]:
if os.path.isfile(os.path.join(model_dir, 'explainer.h5')):
    print('Loading saved model')
    explainer = tf.keras.models.load_model(os.path.join(model_dir, 'explainer.h5'))
    
    fastshap = FastSHAP(explainer = explainer,
                        imputer = surrogate,
                        baseline = 0,
                        normalization=None,
                        link='logit')
    
else:
    fastshap = FastSHAP(explainer = explainer,
                        imputer = surrogate,
                        baseline = 0,
                        normalization=None,
                        link='logit')

    fastshap.train(train_data = ds_train,
                   val_data = ds_val,
                   batch_size = 32,
                   num_samples = 1,
                   max_epochs = 100,
                   validation_batch_size = 32,
                   lr=2e-4,
                   min_lr=1e-5,
                   lr_factor=0.9,
                   eff_lambda=0,
                   paired_sampling=True,
                   lookback=10,
                   verbose=1, 
                   model_dir = model_dir)

Loading saved model


# Plot Explanations

In [17]:
### Function to Reverse Image Transform
def restore_original_image_from_array(x, data_format='channels_last'):
    mean = [103.939, 116.779, 123.68]

    # Zero-center by mean pixel
    if data_format == 'channels_first':
        if x.ndim == 3:
            x[0, :, :] += mean[0]
            x[1, :, :] += mean[1]
            x[2, :, :] += mean[2]
        else:
            x[:, 0, :, :] += mean[0]
            x[:, 1, :, :] += mean[1]
            x[:, 2, :, :] += mean[2]
    else:
        x[..., 0] += mean[0]
        x[..., 1] += mean[1]
        x[..., 2] += mean[2]

    if data_format == 'channels_first':
        # 'BGR'->'RGB'
        if x.ndim == 3:
            x = x[::-1, ...]
        else:
            x = x[:, ::-1, ...]
    else:
        # 'BGR'->'RGB'
        x = x[..., ::-1]

    return x

### Explain Data

In [18]:
for x in ds_test:
    labels = x[1]
    preds = model(x[0])
    images = restore_original_image_from_array(x[0].numpy())
    shap_values = fastshap.shap_values(x[0])
    
    break

### Plot

In [19]:
import matplotlib.pyplot as plt
# plotting code adapted from https://github.com/slundberg/shap

### Plot
class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 
               'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
n_classes = shap_values.shape[-1]
n_images = len(images)
fig_size = np.array([6 * (n_classes + 1), 5 * n_images])
fig, axes = plt.subplots(nrows=n_images, 
                         ncols=n_classes + 1, 
                         figsize=fig_size)
for row in range(len(images)):
    im = images[row]
    
    # get a grayscale version of the image
    if im.max() > 1:
        im /= 255.
    if len(im.shape) == 3 and im.shape[2] == 3:
        im_gray = (0.2989 * im[:,:,0] + 0.5870 * im[:,:,1] + 0.1140 * im[:,:,2]) # rgb to gray
        im_disp = im
    elif len(im.shape) == 3:
        im_gray = im.mean(2)
        # for non-RGB multi-channel data we show an RGB image where each of the three channels is a scaled k-mean center
        flat_vals = im.reshape([im.shape[0]*im.shape[1], im.shape[2]]).T
        flat_vals = (flat_vals.T - flat_vals.mean(1)).T
        means = kmeans(flat_vals, 3, round_values=False).data.T.reshape([im.shape[0], im.shape[1], 3])
        im_disp = (means - np.percentile(means, 0.5, (0,1))) / (np.percentile(means, 99.5, (0,1)) - np.percentile(means, 1, (0,1)))
        im_disp[im_disp > 1] = 1
        im_disp[im_disp < 0] = 0
    else:
        im_gray = im
        im_disp = im
        
    axes[row,0].imshow(im_disp, cmap=plt.get_cmap('gray'))
    axes[row,0].axis('off')
    
    for i in range(n_classes):
        ex = shap_values[row][:,:,i]
        
        #Get Max Val for color range
        abs_vals = np.abs(ex).flatten()
        max_val = np.max(abs_vals) + 1e-7
        y = preds[row][i]
        
        # Title
        if row == 0:
            axes[row,i+1].set_title('{}'.format(class_names[i]), fontsize=62)
        axes[row,i+1].set_xlabel('{:.2f}'.format(y), fontsize=48)
        
        #Plot
        axes[row,i+1].imshow(im_gray, cmap=plt.get_cmap('gray'), alpha=0.3, extent=(-1, ex.shape[1], ex.shape[0], -1))
        im = axes[row,i+1].imshow(ex, cmap='seismic', vmin=-max_val, vmax=max_val)
        
        #Remove Axis
        axes[row,i+1].set_yticklabels([])
        axes[row,i+1].set_xticklabels([])
        axes[row,i+1].set_frame_on(False)
        axes[row,i+1].tick_params(length=0)
 
        
#Figure args
fig.tight_layout()